In [1]:
from pyspark.sql import SparkSession

# stop any existing Spark session, if Spark is already running, creating a new session might fail ***
try:
    spark.stop()
except Exception:
    pass

# create session with adjusted memory settings based on your cluster
# .config("spark.local.dir", r"E:\Apache Spark\spark-temp"): change the spark local dir, as the c disk memory is not enough, may cause Py4JJavaError exception ***
spark = SparkSession.builder.appName("MovieCSVSQL") \
    .config("spark.executor.memory", "32g") \
    .config("spark.driver.memory", "16g") \
    .config("spark.local.dir", r"E:\Apache Spark\spark-temp") \
    .config("spark.jars", r"D:\MySQL\MySQL ConnectorJ\mysql-connector-j-8.4.0\mysql-connector-j-8.4.0\mysql-connector-j-8.4.0.jar") \
    .getOrCreate()
# *** why in the other .ipynb file I can do this, why not here????????, and I change the place, it work again, maybe I should start it all over next time

In [2]:
spark

### Store Movie Data in MySQL

In [3]:
movies = spark.read \
    .format("csv") \
    .option("sep", ",") \
    .option("header", "true") \
    .option("encoding", "utf-8") \
    .option("mode", "DROPMALFORMED") \
    .load("../data/MovieLens 20M Dataset/movie.csv")

In [4]:
movies = movies.withColumnRenamed('movieId', 'movie_id')

In [5]:
movies.show()

+--------+--------------------+--------------------+
|movie_id|               title|              genres|
+--------+--------------------+--------------------+
|       1|    Toy Story (1995)|Adventure|Animati...|
|       2|      Jumanji (1995)|Adventure|Childre...|
|       3|Grumpier Old Men ...|      Comedy|Romance|
|       4|Waiting to Exhale...|Comedy|Drama|Romance|
|       5|Father of the Bri...|              Comedy|
|       6|         Heat (1995)|Action|Crime|Thri...|
|       7|      Sabrina (1995)|      Comedy|Romance|
|       8| Tom and Huck (1995)|  Adventure|Children|
|       9| Sudden Death (1995)|              Action|
|      10|    GoldenEye (1995)|Action|Adventure|...|
|      11|American Presiden...|Comedy|Drama|Romance|
|      12|Dracula: Dead and...|       Comedy|Horror|
|      13|        Balto (1995)|Adventure|Animati...|
|      14|        Nixon (1995)|               Drama|
|      15|Cutthroat Island ...|Action|Adventure|...|
|      16|       Casino (1995)|         Crime|

In [6]:
# Write to MySQL
(movies.write
    .format("jdbc")
    .option("url", "jdbc:mysql://localhost:3307/db_movie_recommender_sys?useSSL=false&serverTimezone=UTC")
    .option("dbtable", "tb_movie")
    .option("user", "root")  # Replace with actual credentials
    .option("password", "137162")
    .option("driver", "com.mysql.cj.jdbc.Driver")
    .mode("overwrite")
    .save())

### Store Rating Data in MySQL

In [7]:
# ratings = spark.read \
#     .format("csv") \
#     .option("sep", ",") \
#     .option("header", "true") \
#     .option("encoding", "utf-8") \
#     .option("mode", "DROPMALFORMED") \
#     .load("../data/MovieLens 20M Dataset/rating.csv")

In [8]:
# rename_dict = {
#     "userId": "user_id",
#     "movieId": "movie_id"
# }


# for old_name, new_name in rename_dict.items():
#     ratings = ratings.withColumnRenamed(old_name, new_name)

In [9]:
# ratings.show()

In [10]:
# ratings = ratings.sample(fraction=0.5, seed=42) # remove half of the data in rating.csv
# ratings = ratings.sample(fraction=0.5, seed=42) # remove half of the data in rating.csv
# ratings = ratings.sample(fraction=0.5, seed=42) # remove half of the data in rating.csv

In [11]:
# print(ratings.count())

In [12]:
# # Write to MySQL
# (ratings.write
#     .format("jdbc")
#     .option("url", "jdbc:mysql://localhost:3307/db_movie_recommender_sys?useSSL=false&serverTimezone=UTC")
#     .option("dbtable", "tb_rating")
#     .option("user", "root")  # Replace with actual credentials
#     .option("password", "137162")
#     .option("driver", "com.mysql.cj.jdbc.Driver")
#     .mode("overwrite")
#     .save())

### Read Movies from MySQL and Analyze

In [13]:
# Read data from MySQL
movies_f_sql = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3307/db_movie_recommender_sys?useSSL=false&serverTimezone=UTC") \
    .option("dbtable", "tb_movie") \
    .option("user", "root") \
    .option("password", "137162") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .load()

In [14]:
movies_f_sql.show()

+--------+--------------------+--------------------+
|movie_id|               title|              genres|
+--------+--------------------+--------------------+
|       1|    Toy Story (1995)|Adventure|Animati...|
|       2|      Jumanji (1995)|Adventure|Childre...|
|       3|Grumpier Old Men ...|      Comedy|Romance|
|       4|Waiting to Exhale...|Comedy|Drama|Romance|
|       5|Father of the Bri...|              Comedy|
|       6|         Heat (1995)|Action|Crime|Thri...|
|       7|      Sabrina (1995)|      Comedy|Romance|
|       8| Tom and Huck (1995)|  Adventure|Children|
|       9| Sudden Death (1995)|              Action|
|      10|    GoldenEye (1995)|Action|Adventure|...|
|      11|American Presiden...|Comedy|Drama|Romance|
|      12|Dracula: Dead and...|       Comedy|Horror|
|      13|        Balto (1995)|Adventure|Animati...|
|      14|        Nixon (1995)|               Drama|
|      15|Cutthroat Island ...|Action|Adventure|...|
|      16|       Casino (1995)|         Crime|